# 20 属性描述符

描述符是实现了特定协议的类，这个协议包括__get__、__set__和__delete__方法。property类实现了完整的描述符协议。

实现了__get__、__set__或__delete__方法的类是描述符。描述符的用法是，创建一个实例，作为另一个类的类属性。

<img src="Chapter20/IMG_E2D17AE68F3D-1.jpeg" alt="drawing" style="width:640px;"/>

## 20.1  描述符示例：验证属性

### 20.1.1 一个简单的描述符 —— 使用基本的描述符类创建属性

In [26]:
# 基本用法

class Quantity: # 描述符类

    def __init__(self, storage_name):
        self.storage_name = storage_name

    def __set__(self, instance, value):
        if value > 0 :
            instance.__dict__[self.storage_name] = value #  这里，必须直接处理托管实例的__dict__属性；如果使用内置的setattr函数，会再次触发__set__方法，导致无限递归。其原因就是托管属性与描述符实例名字相同！
        else:
            raise ValueError('value must be > 0!')

class LineItem: # 托管类
    weight = Quantity('weight') # 描述符实例
    price = Quantity('price')

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight # 托管属性
        self.price = price

    def subtotal(self):
        return self.price * self.weight




apples = LineItem('apple', 2.0, 3.5) # 托管实例

#print(apples.subtotal())

apples.weight = 10.0  # 存储属性


print(apples.subtotal())


print(type(apples.weight))

print(vars(apples))

35.0
<class 'float'>
{'description': 'apple', 'weight': 10.0, 'price': 3.5}


编写__set__方法时，要记住self和instance参数的意思：self是描述符实例，instance是托管实例。管理实例属性的描述符应该把值存储在托管实例中。因此，Python才为描述符中的那个方法提供了instance参数。

```py
# 正确写法：
instance.__dict__[self.storage_name] = value
# 错误写法：
self.__dict__[self.storage_name] = value
```

为了理解错误的原因，可以想想__set__方法前两个参数（self和instance）的意思。这里，self是描述符实例，它其实是托管类的类属性。同一时刻，内存中可能有几千个LineItem实例，不过只会有两个描述符实例：LineItem.weight和LineItem.price。因此，存储在描述符实例中的数据，其实会变成LineItem类的类属性，从而由全部LineItem实例共享。

### 20.1.2 变得更加复杂 —— 自动获取储存属性的名称


**为了避免在描述符声明语句中重复输入属性名，我们将为每个Quantity实例的storage_name属性生成一个独一无二的字符串。**

In [1]:
# 基本用法

class Quantity: # 描述符类
    __count = 0
    def __init__(self):
        cls = self.__class__
        index = cls.__count
        name = cls.__name__
        self.storage_name = '_{}#{}'.format(name, index)
        cls.__count += 1

    def __set__(self, instance, value):
        if value > 0 :
            setattr(instance, self.storage_name, value)
        else:
            raise ValueError('value must be > 0!')

    def __get__(self, instance, name):
        return getattr(instance, self.storage_name)

class LineItem: # 托管类
    weight = Quantity() # 描述符实例
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight # 托管属性
        self.price = price

    def subtotal(self):
        return self.price * self.weight




apples = LineItem('apple', 2.0, 3.5) # 托管实例
banana = LineItem('banana', 1.5, 2) # 托管实例

print(apples.subtotal())

apples.weight = 10.0  # 存储属性


print(apples.subtotal())


print(vars(apples))
print(vars(banana))


7.0
35.0
{'description': 'apple', '_Quantity#0': 10.0, '_Quantity#1': 3.5}
{'description': 'banana', '_Quantity#0': 1.5, '_Quantity#1': 2}


### 20.1.3 一种新型描述符（利用继承）

In [5]:
import abc

class AutoStorage:
    """AutoStorage基类负责自动存储属"""
    _counter = 0
    def __init__(self):
        cls = self.__class__
        prefix = cls.__name__
        index = cls._counter
        self.storage_name = '_{}#{}'.format(prefix, index)
        cls._counter += 1

    def __get__(self, instance, owner):
        if instance  is None:
            return self
        else:
            return getattr(instance, self.storage_name)

    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)

class Validated(abc.ABC, AutoStorage):
    """Validated类做验证"""
    def __set__(self, instance, value):
        value = self.validate(instance, value)
        super().__set__(instance, value)

    @abc.abstractmethod
    def validate(self, instance, value):
        """return validated value or arise ValueError"""

class Quantity(Validated):
    """a number is greater than zero"""
    def validate(self, instance, value):
        if value <= 0:
            raise ValueError("Must be greater than 0")
        return value

class NonBlank(Validated):
    """ a string with at least one non-space character"""
    def validate(self, instance, value):
        value = value.strip() # 去除空格
        if len(value) == 0:
            raise ValueError("value can not be empty or blank")
        return value



class LineItem:
    weight = Quantity()
    price = Quantity()
    description = NonBlank()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.price * self.weight


apples = LineItem('aple', 1.0, 2.0)
apples.weight

LineItem.weight

## 20.2 覆盖型与非覆盖型描述符

- 覆盖型描述符 ：实现了__set__和__get__
- 非覆盖型描述符 ： 只实现了__get__



In [1]:
""" utilities functions """
def cls_name(obj_or_cls):
    cls = type(obj_or_cls)
    if cls is type:
        cls = obj_or_cls
    return cls.__name__.split(".")[-1]

def display(obj):
    cls = type(obj)
    if cls is type:
        return '<class {}>'.format(obj.__name__)
    elif cls in [type(None), int]:
        return repr(obj)
    else:
        return '<{} object>'.format(cls_name(obj))

def print_args(name, *args):
    pseudo_args = ', '.join(display(x) for x in args)
    print("-> {}.__{}__({})".format(cls_name(args[0]), name, pseudo_args))


""" Two types of descriptor """

class Overriding:
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)
    
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

class OverridingNoGet:
    def __set__(self, instance, value):
        print_args('set', self, instance, value)

class NonOverriding:
    def __get__(self, instance, owner):
        print_args('get', self, instance, owner)    


""" Show """
class Managed:
    over = Overriding()
    over_no_get = OverridingNoGet()
    non_over = NonOverriding()

    def spam(self):
        print('-> Managed.spam({})'.format(display(self)))

### 20.2.1 覆盖型描述符

In [2]:
# 覆盖型描述符

obj = Managed()
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [5]:
Managed.over

-> Overriding.__get__(<Overriding object>, None, <class Managed>)


In [7]:
obj.over = 7 # 为obj.over赋值，触发描述符的__set__方法，最后一个参数的值是7。

-> Overriding.__set__(<Overriding object>, <Managed object>, 7)


In [8]:
obj.over

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [11]:
obj.__dict__['over'] = 7
vars(obj)

{'over': 7}

In [10]:
obj.over # 即使是名为over的实例属性，Managed.over描述符仍会覆盖读取obj.over这个操作。

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)


In [18]:
Managed.over = 10 # 覆盖托管类属性，会导致描述符属性被覆盖
Managed.over

10

In [19]:
obj.over # 托管类描述符被覆盖，因此返回了变量中的over值

7

### 20.2.2 没有__get__方法的覆盖型描述符

实例属性会遮盖描述符，不过只有读操作是如此。

In [14]:
obj.over_no_get # 没有get方法，只能返回对象，不能返回值

In [15]:
Managed.over_no_get # 即使托管类也是相同的结果

In [16]:
obj.over_no_get = 8 # 赋值正常。

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 8)


In [20]:
obj.__dict__['over_no_get'] = 10
vars(obj)

{'over': 7, 'over_no_get': 10}

In [21]:
obj.over_no_get # 读取会被同名属性覆盖

10

In [22]:
obj.over_no_get = 12 # 赋值依然采用描述符方法

-> OverridingNoGet.__set__(<OverridingNoGet object>, <Managed object>, 12)


In [23]:
obj.over_no_get # 但是读取时，只要有同名的实例属性，描述符就会被遮盖。

10

### 20.2.3 非覆盖型描述符

如果设置了同名的实例属性，描述符会被遮盖，致使描述符无法处理那个实例的那个属性

方法是以非覆盖型描述符实现的

In [25]:
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


In [26]:
obj.non_over = 3
obj.non_over

3

In [27]:
Managed.non_over

-> NonOverriding.__get__(<NonOverriding object>, None, <class Managed>)


In [28]:
del obj.non_over
obj.non_over

-> NonOverriding.__get__(<NonOverriding object>, <Managed object>, <class Managed>)


### 20.2.4 在类中覆盖描述符

不管描述符是不是覆盖型，为类属性赋值都能覆盖描述

In [29]:
Managed.over = 0
Managed.over_no_get = 1
Managed.non_over = 2
# 不管描述符是不是覆盖型，为类属性赋值都能覆盖描述
Managed.over, Managed.over_no_get, Managed.non_over

(0, 1, 2)

## 20.3 方法是描述符

通过实例访问时，函数的__get__方法返回的是绑定方法对象：一种可调用的对象，里面包装着函数，并把托管实例（例如obj）绑定给函数的第一个参数（即self），这与functools.partial函数的行为一致

绑定方法对象还有个__call__方法，用于处理真正的调用过程。这个方法会调用__func__属性引用的原始函数，把函数的第一个参数设为绑定方法的__self__属性。这就是形参self的隐式绑定方式

函数会变成绑定方法，这是Python语言底层使用描述符的最好例证

In [3]:
obj.spam 
# 通过实例访问时，函数的__get__方法返回的是绑定方法对象：一种可调用的对象，里面包装着函数，并把托管实例（例如obj）绑定给函数的第一个参数（即self），
# 这与functools.partial函数的行为一致

<bound method Managed.spam of <__main__.Managed object at 0x7f8507cda9d0>>

In [4]:
obj.spam()

-> Managed.spam(<Managed object>)


In [5]:
Managed.spam

<function __main__.Managed.spam(self)>

In [6]:
obj.spam = 7
obj.spam

7

In [8]:
Managed.spam

<function __main__.Managed.spam(self)>

In [9]:
obj.spam() # 如果为obj.spam赋值，会遮盖类属性，导致无法通过obj实例访问spam方法。

TypeError: 'int' object is not callable

In [10]:
# 类成员函数的本质
from collections import UserString

class Text(UserString):
    def __repr__(self):
        return 'Text({!r})'.format(self.data)
    def reverse(self):
        return self[::-1]



In [11]:
word = Text('forward')
word

Text('forward')

In [13]:
word.reverse()

Text('drawrof')

In [14]:
word.reverse

<bound method Text.reverse of Text('forward')>

In [15]:
Text.reverse

<function __main__.Text.reverse(self)>

In [16]:
Text.reverse()

TypeError: reverse() missing 1 required positional argument: 'self'

In [17]:
Text.reverse(Text('drawrof')) #  在类上调用方法相当于调用函

Text('forward')

In [18]:
type(word.reverse), type(Text.reverse) # 注意类型是不同的，一个是function，一个是method。

(method, function)

In [19]:
list(map(Text.reverse, ['repaid', (10,20,30), Text('backward')])) #  Text.reverse相当于函数，甚至可以处理Text实例之外的其他对象

['diaper', (30, 20, 10), Text('drawkcab')]

In [21]:
Text.reverse.__get__(word) #  函数都是非覆盖型描述符。在函数上调用__get__方法时传入实例，得到的是绑定到那个实例上的方法

<bound method Text.reverse of Text('forward')>

In [24]:
Text.reverse.__get__(None, Text) # 调用函数的__get__方法时，如果instance参数的值是None，那么得到的是函数本身。

<function __main__.Text.reverse(self)>

In [26]:
word.reverse.__self__ #  绑定方法对象有个__self__属性，其值是调用这个方法的实例引

Text('forward')

In [28]:
word.reverse.__func__ is Text.reverse # 绑定方法的__func__属性是依附在托管类上那个原始函数的引用

True

函数会变成绑定方法，这是Python语言底层使用描述符的最好例证

## 20.4 描述符用法建议

- 使用特性以保持简
- 只读描述符必须有__set__方
- 用于验证的描述符可以只有__set__方法
- 仅有__get__方法的描述符可以实现高效缓
- 非特殊的方法可以被实例属性遮盖